In [12]:
import pandas as pd

# Load Excel files
kundendatei_path = 'Kundendatei.xlsx'
ep_katalog_path = 'EP_Katalog.xlsx'

# Load sheets - assume first sheet for now
customer_df = pd.read_excel(kundendatei_path)
ep_catalog_df = pd.read_excel(ep_katalog_path)

# Show shapes and head
print("Customer file shape:", customer_df.shape)
print("EP catalog shape:", ep_catalog_df.shape)

customer_df.head(3)


Customer file shape: (973, 61)
EP catalog shape: (695, 7)


,WirtEinh,EQ_übergeordnet,Equipment,EQ-Bezeichnung,EQ-Klasse,EQ-Klasse-Bezeichnung,Gewerk,Anlagenausprägung,EQ-Menge,EQ-Merkmal_001,...,EQ-Merkmal_043,EQ-Merkmal_044,EQ-Merkmal_045,EQ-Merkmal_046,EQ-Merkmal_047,EQ-Merkmal_048,EQ-Merkmal_049,EQ-Merkmal_050,EQ-Merkmal_051,EQ-Merkmal_052
0,21083,NaN,1000005794,Schutzraumlüftung - Mitt.(Außer Betrieb),31211,Lüftungsanlagen,RLT,Schutzraumlüft. ohne Sandfilt.,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21083,NaN,1000005932,RLT Batterieraum alt - Mittelbau,31211,Lüftungsanlagen,RLT,Zu-/Abluftanlage,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21083,NaN,1000005934,RLT Gleichr. alt - Mittelbau,31211,Lüftungsanlagen,RLT,Zu-/Abluftanlage,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## EDA

In [13]:
# Show column names to check for typos or whitespace
print("Customer Columns:")
print(customer_df.columns.tolist())

print("\nEP Catalog Columns:")
print(ep_catalog_df.columns.tolist())

Customer Columns:
['WirtEinh', 'EQ_übergeordnet', 'Equipment', 'EQ-Bezeichnung', 'EQ-Klasse', 'EQ-Klasse-Bezeichnung', 'Gewerk', 'Anlagenausprägung', 'EQ-Menge', 'EQ-Merkmal_001', 'EQ-Merkmal_002', 'EQ-Merkmal_003', 'EQ-Merkmal_004', 'EQ-Merkmal_005', 'EQ-Merkmal_006', 'EQ-Merkmal_007', 'EQ-Merkmal_008', 'EQ-Merkmal_009', 'EQ-Merkmal_010', 'EQ-Merkmal_011', 'EQ-Merkmal_012', 'EQ-Merkmal_013', 'EQ-Merkmal_014', 'EQ-Merkmal_015', 'EQ-Merkmal_016', 'EQ-Merkmal_017', 'EQ-Merkmal_018', 'EQ-Merkmal_019', 'EQ-Merkmal_020', 'EQ-Merkmal_021', 'EQ-Merkmal_022', 'EQ-Merkmal_023', 'EQ-Merkmal_024', 'EQ-Merkmal_025', 'EQ-Merkmal_026', 'EQ-Merkmal_027', 'EQ-Merkmal_028', 'EQ-Merkmal_029', 'EQ-Merkmal_030', 'EQ-Merkmal_031', 'EQ-Merkmal_032', 'EQ-Merkmal_033', 'EQ-Merkmal_034', 'EQ-Merkmal_035', 'EQ-Merkmal_036', 'EQ-Merkmal_037', 'EQ-Merkmal_038', 'EQ-Merkmal_039', 'EQ-Merkmal_040', 'EQ-Merkmal_041', 'EQ-Merkmal_042', 'EQ-Merkmal_043', 'EQ-Merkmal_044', 'EQ-Merkmal_045', 'EQ-Merkmal_046', 'EQ-Merkma

In [19]:
import re

def extract_kostengruppe(merkmal):
    if pd.isnull(merkmal):
        return None
    match = re.search(r'Kostengruppe DIN 276-1:\s*(\d{3})', str(merkmal))
    if match:
        return match.group(1)
    return None


# Look through all EQ-Merkmal columns to find a Kostengruppe
def search_kostengruppe_in_all(row):
    for col in row.index:
        if 'EQ-Merkmal' in col:
            match = extract_kostengruppe(row[col])
            if match:
                return match
    return None

# Apply row-wise
customer_df['Extracted_Kostengruppe'] = customer_df.apply(search_kostengruppe_in_all, axis=1)

# Check again
customer_df[['EQ-Bezeichnung', 'Extracted_Kostengruppe']].head(10)


,EQ-Bezeichnung,Extracted_Kostengruppe
0,Schutzraumlüftung - Mitt.(Außer Betrieb),431
1,RLT Batterieraum alt - Mittelbau,431
2,RLT Gleichr. alt - Mittelbau,431
3,"Gebäude-Blitzschutz-, Erdungsanl., Pot.",446
4,Beschallungs-/Sprechanlagen,454
5,"WTA Entwässerungspumpen, ortsfest",411
6,Niederspannungsleitungsanlage,444
7,"Innenbeleuchtung, ortsfest",445
8,Wasserverteilanlage,412
9,Laufkatzenanlage,465


In [20]:
has_kgr = customer_df['Extracted_Kostengruppe'].notnull().sum()
no_kgr = customer_df['Extracted_Kostengruppe'].isnull().sum()

print(f"✅ Found Kostengruppe for {has_kgr} systems")
print(f"❌ Missing Kostengruppe for {no_kgr} systems")

✅ Found Kostengruppe for 913 systems
❌ Missing Kostengruppe for 60 systems


In [23]:

def extract_ep_group_prefix(artnr):
    if pd.isnull(artnr):
        return None
    match = re.match(r'(\d{3})', str(artnr))
    return match.group(1) if match else None

# Drop rows with missing Artikelnummer — these are just catalog section headers
ep_items_df = ep_catalog_df.dropna(subset=['Artikelnummer']).copy()

# Re-extract group codes just on cleaned data
ep_items_df['GroupCode'] = ep_items_df['Artikelnummer'].apply(extract_ep_group_prefix)


# Let's see how many unique groups exist
print("Unique EP Catalog Group Codes:", ep_items_df['GroupCode'].nunique())


Unique EP Catalog Group Codes: 43


In [24]:
# Now you should see only actual items
ep_items_df[['Kurztext / Bezeichnung', 'Artikelnummer', 'GroupCode']].head(10)

,Kurztext / Bezeichnung,Artikelnummer,GroupCode
3,334 - Fenster mit Kipp- oder Drehflügel - kraf...,334.01.01.000.01.00,334
4,334 - Fenster mit Kipp- oder Drehflügel - nich...,334.01.01.000.02.00,334
5,334 - Hochwasserschutzfenster - nicht kraftbet...,334.01.02.000.01.00,334
6,334 - Falttor - kraftbetrieben - Breite < 8 m;...,334.02.01.000.01.01,334
7,334 - Falttor - nicht kraftbetrieben - Breite ...,334.02.01.000.02.01,334
8,334 - Rolltore / Rollgittertore - kraftbetrieb...,334.02.01.000.03.01,334
9,334 - Rolltore / Rollgittertore - nicht kraftb...,334.02.01.000.04.01,334
10,334 - Schiebetore (inkl. Feststellanlage) - kr...,334.02.01.000.05.01,334
11,334 - Schiebetore (inkl. Feststellanlage) - ni...,334.02.01.000.06.01,334
12,334 - Schiebetore (inkl. Feststellanlage) - kr...,334.02.01.000.07.01,334


In [27]:
import json
import re
import pandas as pd
from collections import defaultdict

class CatalogueParser:
    def __init__(self):
        self.tree = defaultdict(dict)
        
    def parse_catalogue_dataframe(self, df, description_col=None, article_col=None):
        """
        Parse the catalogue data from pandas DataFrame
        
        Args:
            df (pd.DataFrame): DataFrame containing catalogue data
            description_col (str): Name of the description column (auto-detected if None)
            article_col (str): Name of the article number column (auto-detected if None)
        
        Returns:
            dict: Hierarchical tree structure
        """
        # Auto-detect column names if not provided
        if description_col is None:
            # Look for common description column names
            desc_candidates = ['Kurztext / Bezeichnung', 'Kurztext', 'Bezeichnung', 'Description', 'Item']
            for col in df.columns:
                if any(candidate.lower() in col.lower() for candidate in desc_candidates):
                    description_col = col
                    break
            if description_col is None:
                description_col = df.columns[0]  # Use first column as fallback
        
        if article_col is None:
            # Look for common article number column names
            article_candidates = ['Artikelnummer', 'Article', 'Number', 'Code', 'ID']
            for col in df.columns:
                if any(candidate.lower() in col.lower() for candidate in article_candidates):
                    article_col = col
                    break
            if article_col is None:
                article_col = df.columns[1] if len(df.columns) > 1 else df.columns[0]
        
        print(f"Using columns: Description='{description_col}', Article='{article_col}'")
        
        # Process each row
        for idx, row in df.iterrows():
            description = str(row[description_col]).strip() if pd.notna(row[description_col]) else ""
            article_number = str(row[article_col]).strip() if pd.notna(row[article_col]) else ""
            
            # Skip empty rows, headers, and rows without article numbers
            if (description and article_number and 
                article_number != 'nan' and 
                not description.startswith('Wartung') and
                not description.lower().startswith('kurztext')):
                
                self.parse_item(description, article_number)
        
        return dict(self.tree)
    
    def parse_catalogue_data(self, data_text):
        """
        Parse the catalogue data from text format (kept for backward compatibility)
        
        Args:
            data_text (str): Raw catalogue data as text
        
        Returns:
            dict: Hierarchical tree structure
        """
        lines = data_text.strip().split('\n')
        
        for line in lines:
            if '\t' in line and line.strip():
                # Split by tab to separate description and article number
                parts = line.strip().split('\t')
                if len(parts) >= 2:
                    description = parts[0].strip()
                    article_number = parts[1].strip() if parts[1].strip() else None
                    
                    # Skip header lines and category headers without article numbers
                    if description and article_number and not description.startswith('Wartung'):
                        self.parse_item(description, article_number)
        
        return dict(self.tree)
    
    def parse_item(self, description, article_number):
        """
        Parse individual item description and create hierarchical structure
        
        Args:
            description (str): Item description
            article_number (str): Article number
        """
        # Remove leading numbers and dashes (e.g., "334 - ")
        cleaned_desc = re.sub(r'^\d+\s*-\s*', '', description).strip()
        
        # Split by " - " to get different levels
        parts = [part.strip() for part in cleaned_desc.split(' - ')]
        
        if not parts:
            return
        
        # First part is usually the main category
        main_category = parts[0]
        
        # Process the rest of the description
        if len(parts) > 1:
            # Join the remaining parts and parse them
            detail_description = ' - '.join(parts[1:])
            hierarchy = self.extract_hierarchy(detail_description)
        else:
            hierarchy = []
        
        # Build the tree structure
        self.build_tree_path(main_category, hierarchy, article_number)
    
    def extract_hierarchy(self, description):
        """
        Extract hierarchical information from item description
        
        Args:
            description (str): Detailed description
        
        Returns:
            list: List of hierarchical levels
        """
        hierarchy = []
        
        # Common patterns to identify different levels
        patterns = {
            'type': r'^([^-;]+?)(?:\s*-\s*|\s*;\s*|$)',
            'power': r'(kraftbetrieben|nicht kraftbetrieben|elektrisch|mechanisch)',
            'function': r'(mit|ohne)\s+(Brand-?/?Rauchschutzfunktion|Fluchtwegsfunktion|Brandschutzfunktion)',
            'wings': r'(\d+-flüglig)',
            'size': r'(Breite\s*[<>]\s*\d+\s*m[^;]*|Höhe\s*[<>]\s*\d+\s*m[^;]*|Einbauhöhe[^;]*|Innendurchmesser[^;]*|Dachfläche[^;]*)',
        }
        
        # Extract main type/category
        type_match = re.search(patterns['type'], description)
        if type_match:
            main_type = type_match.group(1).strip()
            if main_type and not any(x in main_type.lower() for x in ['kraftbetrieben', 'elektrisch', 'mechanisch']):
                hierarchy.append(('type', main_type))
        
        # Extract power/operation method
        power_match = re.search(patterns['power'], description)
        if power_match:
            hierarchy.append(('power', power_match.group(1)))
        
        # Extract function (Brand/Rauch/Fluchtweg)
        function_match = re.search(patterns['function'], description)
        if function_match:
            function_desc = f"{function_match.group(1)} {function_match.group(2)}"
            hierarchy.append(('function', function_desc))
        
        # Extract wings information
        wings_match = re.search(patterns['wings'], description)
        if wings_match:
            hierarchy.append(('wings', wings_match.group(1)))
        
        # Extract size information
        size_match = re.search(patterns['size'], description)
        if size_match:
            hierarchy.append(('size', size_match.group(1).strip()))
        
        # Handle special cases
        if 'Zulage' in description:
            hierarchy.append(('type', 'Zubehör'))
            if 'Feststellanlage' in description:
                hierarchy.append(('subtype', 'Feststellanlage für Brand-/Rauchschutzfunktion'))
            elif 'Motorschloss' in description:
                hierarchy.append(('subtype', 'Motorschloss'))
            elif 'Flichttürterminal' in description:
                hierarchy.append(('subtype', 'Flichttürterminal'))
            elif 'Fluchtürwächter' in description:
                hierarchy.append(('subtype', 'Fluchtürwächter'))
        
        return hierarchy
    
    def build_tree_path(self, main_category, hierarchy, article_number):
        """
        Build the tree path from main category through hierarchy to article number
        
        Args:
            main_category (str): Main category
            hierarchy (list): List of (type, value) tuples
            article_number (str): Final article number
        """
        # Ensure main category exists as a dict
        if main_category not in self.tree:
            self.tree[main_category] = {}
        elif not isinstance(self.tree[main_category], dict):
            self.tree[main_category] = {}
        
        current = self.tree[main_category]
        
        # Group hierarchy items by logical structure
        type_items = [item[1] for item in hierarchy if item[0] == 'type']
        power_items = [item[1] for item in hierarchy if item[0] == 'power']
        function_items = [item[1] for item in hierarchy if item[0] == 'function']
        wings_items = [item[1] for item in hierarchy if item[0] == 'wings']
        size_items = [item[1] for item in hierarchy if item[0] == 'size']
        subtype_items = [item[1] for item in hierarchy if item[0] == 'subtype']
        
        # Build path: type -> power -> function -> wings -> size
        path_elements = type_items + power_items + function_items + wings_items + size_items + subtype_items
        
        # Navigate/create the path
        for i, element in enumerate(path_elements):
            if element not in current:
                current[element] = {}
            elif not isinstance(current[element], dict):
                # If we hit a leaf node (string), we need to restructure
                # Save the existing value and convert to dict
                existing_value = current[element]
                current[element] = {"_direct": existing_value}
            current = current[element]
        
        # Set the article number at the end
        if path_elements:
            # Navigate back to set the final value
            target = self.tree[main_category]
            for element in path_elements[:-1]:
                target = target[element]
            
            # Check if the final element already exists
            final_element = path_elements[-1]
            if final_element in target and isinstance(target[final_element], dict):
                # If it's a dict, add the article number as a special key
                target[final_element]["_article"] = article_number
            else:
                # Direct assignment
                target[final_element] = article_number
        else:
            # Direct assignment to main category - but main category should be a dict
            # Store the article number with a special key
            if isinstance(self.tree[main_category], dict):
                self.tree[main_category]["_direct"] = article_number
            else:
                self.tree[main_category] = article_number
    
    def save_to_json(self, filename, tree_data):
        """Save the tree structure to a JSON file"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(tree_data, f, indent=2, ensure_ascii=False)
    
    def load_from_file(self, filename):
        """Load catalogue data from file"""
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()

class CatalogueDecisionTree:
    def __init__(self, tree_data=None):
        self.tree = tree_data or {}
    
    def load_from_json(self, filename):
        """Load tree structure from JSON file"""
        with open(filename, 'r', encoding='utf-8') as f:
            self.tree = json.load(f)
    
    def get_article_number(self, path):
        """
        Navigate through the decision tree using a list of choices
        
        Args:
            path (list): List of strings representing the path through the tree
        
        Returns:
            str: Article number if found, None otherwise
        """
        current = self.tree
        
        for choice in path:
            if isinstance(current, dict) and choice in current:
                current = current[choice]
            else:
                return None
        
        # Handle special keys for article numbers
        if isinstance(current, str):
            return current
        elif isinstance(current, dict):
            if "_article" in current:
                return current["_article"]
            elif "_direct" in current:
                return current["_direct"]
        
        return None
    
    def get_available_options(self, path=None):
        """
        Get available options at the current level of the tree
        
        Args:
            path (list): Current path in the tree (optional)
        
        Returns:
            list: Available options at the current level
        """
        if path is None:
            return list(self.tree.keys())
        
        current = self.tree
        for choice in path:
            if isinstance(current, dict) and choice in current:
                current = current[choice]
            else:
                return []
        
        if isinstance(current, dict):
            # Filter out special keys that start with underscore
            return [key for key in current.keys() if not key.startswith('_')]
        else:
            return []
    
    def find_article_numbers(self, search_term):
        """
        Search for article numbers containing specific terms
        
        Args:
            search_term (str): Term to search for
        
        Returns:
            list: List of (path, article_number) tuples
        """
        results = []
        
        def search_recursive(node, current_path):
            if isinstance(node, str):
                # This is an article number
                if any(search_term.lower() in part.lower() for part in current_path):
                    results.append((current_path.copy(), node))
            elif isinstance(node, dict):
                for key, value in node.items():
                    if key.startswith('_'):
                        # Handle special keys
                        if key == "_article" or key == "_direct":
                            if any(search_term.lower() in part.lower() for part in current_path):
                                results.append((current_path.copy(), value))
                    else:
                        search_recursive(value, current_path + [key])
        
        search_recursive(self.tree, [])
        return results
    
    def print_tree_structure(self, max_depth=3):
        """Print the tree structure for debugging"""
        def print_recursive(node, depth=0, prefix=""):
            if depth > max_depth:
                return
            
            if isinstance(node, str):
                print(f"{prefix}→ {node}")
            elif isinstance(node, dict):
                for i, (key, value) in enumerate(node.items()):
                    is_last = i == len(node) - 1
                    print(f"{prefix}{'└── ' if is_last else '├── '}{key}")
                    extension = "    " if is_last else "│   "
                    print_recursive(value, depth + 1, prefix + extension)
        
        print_recursive(self.tree)

# Example usage and testing
if __name__ == "__main__":
    
    # Method 1: Using pandas DataFrame (recommended for Excel files)
    ep_katalog_path = 'EP_Katalog.xlsx'
    
    try:
        # Read Excel file
        ep_catalog_df = pd.read_excel(ep_katalog_path)
        print("Excel file loaded successfully!")
        print(f"Shape: {ep_catalog_df.shape}")
        print(f"Columns: {ep_catalog_df.columns.tolist()}")
        print("\nFirst few rows:")
        print(ep_catalog_df.head())
        
        # Parse the catalogue from DataFrame
        parser = CatalogueParser()
        tree_data = parser.parse_catalogue_dataframe(ep_catalog_df)
        
        # Save to JSON file
        parser.save_to_json('catalogue_tree.json', tree_data)
        print("\nTree structure saved to 'catalogue_tree.json'")
        
    except FileNotFoundError:
        print(f"File {ep_katalog_path} not found. Using sample data instead.")
        
        # Method 2: Using sample text data (fallback)
        catalogue_data = """Kurztext / Bezeichnung	Artikelnummer
Wartung und Inspektion	
330 - Außenwände - Wände und Stützen, die dem Außenklima ausgesetzt sind bzw. an das Erdreich oder an andere Bauwerke grenzen	
334 - Außentüren und -fenster	
334 - Fenster mit Kipp- oder Drehflügel - kraftbetrieben	334.01.01.000.01.00
334 - Fenster mit Kipp- oder Drehflügel - nicht kraftbetrieben	334.01.01.000.02.00
334 - Hochwasserschutzfenster - nicht kraftbetrieben	334.01.02.000.01.00
334 - Falttor - kraftbetrieben - Breite < 8 m; Höhe <6 m	334.02.01.000.01.01
334 - Falttor - nicht kraftbetrieben - Breite < 8 m; Höhe <6 m	334.02.01.000.02.01
338 - Sonnenschutz - Rollläden, Markisen und Jalousien einschließlich Antrieben	
338 - Rolläden - elektrisch	338.01.01.194.01.00
338 - Rolläden - mechanisch	338.01.01.194.02.00"""
        
        # Parse the catalogue from text
        parser = CatalogueParser()
        tree_data = parser.parse_catalogue_data(catalogue_data)
        
        # Save to JSON file
        parser.save_to_json('catalogue_tree.json', tree_data)
    
    # Create decision tree instance
    decision_tree = CatalogueDecisionTree(tree_data)
    
    # Print the structure
    print("\nGenerated tree structure:")
    decision_tree.print_tree_structure()
    
    # Example searches
    print("\nExample usage:")
    print("Available main categories:", decision_tree.get_available_options())
    
    # Search for a specific item
    print("\nSearching for 'Fenster':")
    results = decision_tree.find_article_numbers('Fenster')
    for path, article_num in results:
        print(f"  Path: {' -> '.join(path)} | Article: {article_num}")
    
    # Navigate to specific item
    print("\nNavigating through tree:")
    main_cats = decision_tree.get_available_options()
    if main_cats:
        first_cat = main_cats[0]
        print(f"Options under '{first_cat}':", decision_tree.get_available_options([first_cat]))

Excel file loaded successfully!
Shape: (695, 7)
Columns: ['Unnamed: 0', 'OZ', 'Art', 'Menge', 'Einheit', 'Kurztext / Bezeichnung', 'Artikelnummer']

First few rows:
                  Unnamed: 0            OZ                 Art  Menge Einheit  \
0      E - Bedarfspos. o. GB           01.   NG - Normalgruppe    NaN     NaN   
1     A - Alternativposition        01.01.   NG - Normalgruppe    NaN     NaN   
2          G - Grundposition     01.01.01.   NG - Normalgruppe    NaN     NaN   
3         B - Bezugsposition  01.01.01.01.  N - Normalposition    1.0     Stk   
4  W - Wiederholungsposition  01.01.01.02.  N - Normalposition    1.0     Stk   

                              Kurztext / Bezeichnung        Artikelnummer  
0                             Wartung und Inspektion                  NaN  
1  330 - Außenwände - Wände und Stützen, die dem ...                  NaN  
2                      334 - Außentüren und -fenster                  NaN  
3  334 - Fenster mit Kipp- oder Drehflügel -

│   ├── Stromkreise 20 bis 50 Stk.
│   │   └── _article
│   │       → 444.03.01.000.00.02
│   └── Stromkreise 50 bis 100 Stk.
│       └── _article
│           → 444.03.01.000.00.03
├── NS-Unterverteiler
│   └── Zulage Stromzähler
│       └── Zubehör
│           └── _article
├── Verlegesysteme
│   └── _direct
│       → 444.04.01.000.00.00
├── Installationsgeräte
│   └── _direct
│       → 444.05.01.000.00.00
├── Ortsfeste Leuchten
│   └── _direct
│       → 445.01.01.000.00.00
├── Sicherheitsbeleuchtungsanlage
│   ├── Leuchte
│   │   └── _article
│   │       → 445.02.01.007.04.00
│   └── Zentralgerät
│       └── _article
│           → 445.02.01.235.00.00
├── optische Sicherheitsleitsysteme
│   ├── Sicherheitszeichen
│   │   └── _article
│   │       → 445.02.02.200.02.00
│   └── Leitmarkierungen
│       └── _article
│           → 445.02.02.174.00.00
├── Ampelanlage
│   └── _direct
│       → 445.03.01.000.00.00
├── Äußerere Blitzschutz (inkl. Auffangeinrichtungen, Ableitungen, Erdungen)
│  

### Customer File (Kundendatei)

In [3]:
# Basic info
# print(customer_df.info())
# print(customer_df.describe(include='all'))

# Count of buildings
print("Number of unique buildings:", customer_df['WirtEinh'].nunique())

# Top system/eq types
print("Top systems/equipments (EQ-Bezeichnung):")
print(customer_df['EQ-Bezeichnung'].value_counts().head())


# Top equipment classes
print("Top EQ-Klasse-Bezeichnung:")
print(customer_df['EQ-Klasse-Bezeichnung'].value_counts().head())


# Systems/eq per building
systems_per_building = customer_df.groupby('WirtEinh')['Equipment'].nunique()
print("Average systems/equipments per building:", systems_per_building.mean())


Number of unique buildings: 7
Top systems/equipments (EQ-Bezeichnung):
Innenbeleuchtung, ortsfest       17
Abwasserleitungsanlage           15
Feuerlöscher                     15
Wasserverteilanlage              13
Niederspannungsleitungsanlage    10
Name: EQ-Bezeichnung, dtype: int64
Top EQ-Klasse-Bezeichnung:
IZS-Komponenten                   342
IZS-Zutrittskontrollzentrale       55
Niederspannungsleitungsanlagen     42
Lüftungsanlagen                    38
Kleinlöschgeräte                   28
Name: EQ-Klasse-Bezeichnung, dtype: int64
Average systems/equipments per building: 139.0


### Sample Objects (Beispielobjekte)

In [4]:
# print(sample_df.info())
# print(sample_df.describe(include='all'))

# Number of buildings
print("Unique buildings (sample):", sample_df['Gebäude-ID'].nunique())

# Most common systems
print("Top systems (Anlagenname):")
print(sample_df['Anlagenname'].value_counts().head(10))

# Systems per building
systems_per_building_ref = sample_df.groupby('Gebäude-ID')['Anlagenname'].nunique()
print("Average systems per building (sample):", systems_per_building_ref.mean())


Unique buildings (sample): 32
Top systems (Anlagenname):
Außentür                         543
Handfeuerlöscher                 350
Pumpen                           199
Stromzähler                      181
GA-Schaltschränke                140
NS-Unterverteiler                 86
Wärmemengenzähler                 78
Membranausdehnungsgefäß (MAG)     61
Niederspannungshauptverteiler     54
Abläufe                           49
Name: Anlagenname, dtype: int64
Average systems per building (sample): 151.09375


### EP Katalog

In [ ]:
# print(ep_catalog_df.columns)

In [6]:
# print(ep_catalog_df.info())
# print(ep_catalog_df.describe(include='all'))

# Most common system types (by normalized name)
# print("Top catalog entries:")
# print(ep_catalog_df['Kurztext / Bezeichnung'].value_counts().head(10))

# Count unique catalog items
print("Unique item numbers:", ep_catalog_df['Artikelnummer'].nunique())


Unique item numbers: 614


#### Grouping Structure of Our Catalogue

In [38]:
filtered_df = ep_catalog_df[
    (ep_catalog_df['Art'] == 'NG - Normalgruppe') & 
    (ep_catalog_df['Artikelnummer'].fillna('').str.strip() == '')
]
# Select the corresponding values from another column
result = filtered_df['Kurztext / Bezeichnung']

# Show result
print(result)
result.to_csv('result.txt', index=False, header=False)

0                                 Wartung und Inspektion
1      330 - Außenwände - Wände und Stützen, die dem ...
2                          334 - Außentüren und -fenster
64     338 - Sonnenschutz - Rollläden, Markisen und J...
73     339 - Außenwände, sonstiges - Gitter, Geländer...
75        340 - Innenwände - Innenwände und Innenstützen
76     346 - Elementierte Innenwände - Elementierte W...
79            360 - Dächer - Flache oder geneigte Dächer
80     362 - Dachfenster, Dachöffnungen - Fenster, Au...
83     363 - Dachbeläge - Beläge auf Dachkonstruktion...
89            380 - Baukonstruktive Einbauten, sonstiges
90     387 - Baukonstruktive Einbauten, sonstiges - z...
99     390 - Sonstige Maßnahmen für Baukonstruktionen...
100    399 - Sonstige Maßnahmen für Baukonstruktionen...
107                 410 - Abwasser-, Wasser-, Gasanlagen
108                                411 - Abwasseranlagen
142                                  412 - Wasseranlagen
174                            

In [39]:
kostengruppe_mapping = {}

for text in result:
    key = text.split(' - ')[0].strip()  # Extract leading number
    kostengruppe_mapping[key] = [text]  # Full text as list

print(kostengruppe_mapping)

{'Wartung und Inspektion': ['Wartung und Inspektion'], '330': ['330 - Außenwände - Wände und Stützen, die dem Außenklima ausgesetzt sind bzw. an das Erdreich oder an andere Bauwerke grenzen'], '334': ['334 - Außentüren und -fenster'], '338': ['338 - Sonnenschutz - Rollläden, Markisen und Jalousien einschließlich Antrieben'], '339': ['339 - Außenwände, sonstiges - Gitter, Geländer, Stoßabweiser und Handläufe'], '340': ['340 - Innenwände - Innenwände und Innenstützen'], '346': ['346 - Elementierte Innenwände - Elementierte Wände, bestehend aus Innenwänden, -türen, -fenstern, -bekleidungen, z.\xa0B. Falt- und Schiebewände, Sanitärtrennwände, Verschläge'], '360': ['360 - Dächer - Flache oder geneigte Dächer'], '362': ['362 - Dachfenster, Dachöffnungen - Fenster, Ausstiege einschließlich Umrahmungen, Beschlägen, Antrieben, Lüftungselementen und sonstigen eingebauten Elementen'], '363': ['363 - Dachbeläge - Beläge auf Dachkonstruktionen einschließlich Schalungen, Lattungen, Gefälle-, Dichtun

### Quick Try To See the State of the Problem

In [41]:
import json
import re
import pandas as pd
from collections import defaultdict

# Kostengruppe mapping - adjust later
kostengruppe_mapping = {
    '334': ['Fensterbänke', 'Fenster', 'Türen'],
    '335': ['Türen', 'Tore'],
    '336': ['Rollläden', 'Jalousien', 'Markisen'],
    '421': ['Abwasseranlagen'],
    '422': ['Wasseranlagen'],
    '431': ['Wärmeversorgungsanlagen'],
    '432': ['Raumlufttechnische Anlagen'],
    '441': ['Starkstromanlagen'],
    '442': ['Fernmelde- und informationstechnische Anlagen'],
    '443': ['Förderanlagen'],
    '444': ['Nutzungsspezifische Anlagen'],
    '445': ['Gebäudeautomation'],
    '446': ['Anlagen der Außenanlagen']
}

class CatalogueDecisionTree:
    def __init__(self, tree_data=None):
        self.tree = tree_data or {}
    
    def load_from_json(self, filename):
        """Load tree structure from JSON file"""
        with open(filename, 'r', encoding='utf-8') as f:
            self.tree = json.load(f)
    
    def get_article_number(self, path):
        """
        Navigate through the decision tree using a list of choices
        
        Args:
            path (list): List of strings representing the path through the tree
        
        Returns:
            str: Article number if found, None otherwise
        """
        current = self.tree
        
        for choice in path:
            if isinstance(current, dict) and choice in current:
                current = current[choice]
            else:
                return None
        
        # Handle special keys for article numbers
        if isinstance(current, str):
            return current
        elif isinstance(current, dict):
            if "_article" in current:
                return current["_article"]
            elif "_direct" in current:
                return current["_direct"]
        
        return None
    
    def get_available_options(self, path=None):
        """
        Get available options at the current level of the tree
        
        Args:
            path (list): Current path in the tree (optional)
        
        Returns:
            list: Available options at the current level
        """
        if path is None:
            return list(self.tree.keys())
        
        current = self.tree
        for choice in path:
            if isinstance(current, dict) and choice in current:
                current = current[choice]
            else:
                return []
        
        if isinstance(current, dict):
            # Filter out special keys that start with underscore
            return [key for key in current.keys() if not key.startswith('_')]
        else:
            return []
    
    def find_article_numbers(self, search_term):
        """
        Search for article numbers containing specific terms
        
        Args:
            search_term (str): Term to search for
        
        Returns:
            list: List of (path, article_number) tuples
        """
        results = []
        
        def search_recursive(node, current_path):
            if isinstance(node, str):
                # This is an article number
                if any(search_term.lower() in part.lower() for part in current_path):
                    results.append((current_path.copy(), node))
            elif isinstance(node, dict):
                for key, value in node.items():
                    if key.startswith('_'):
                        # Handle special keys
                        if key == "_article" or key == "_direct":
                            if any(search_term.lower() in part.lower() for part in current_path):
                                results.append((current_path.copy(), value))
                    else:
                        search_recursive(value, current_path + [key])
        
        search_recursive(self.tree, [])
        return results
    
    def print_tree_structure(self, max_depth=3):
        """Print the tree structure for debugging"""
        def print_recursive(node, depth=0, prefix=""):
            if depth > max_depth:
                return
            
            if isinstance(node, str):
                print(f"{prefix}→ {node}")
            elif isinstance(node, dict):
                for i, (key, value) in enumerate(node.items()):
                    is_last = i == len(node) - 1
                    print(f"{prefix}{'└── ' if is_last else '├── '}{key}")
                    extension = "    " if is_last else "│   "
                    print_recursive(value, depth + 1, prefix + extension)
        
        print_recursive(self.tree)

class CatalogueParser:
    def __init__(self):
        self.tree = defaultdict(dict)
        
    def parse_catalogue_dataframe(self, df, description_col=None, article_col=None):
        """
        Parse the catalogue data from pandas DataFrame
        
        Args:
            df (pd.DataFrame): DataFrame containing catalogue data
            description_col (str): Name of the description column (auto-detected if None)
            article_col (str): Name of the article number column (auto-detected if None)
        
        Returns:
            dict: Hierarchical tree structure
        """
        # Auto-detect column names if not provided
        if description_col is None:
            # Look for common description column names
            desc_candidates = ['Kurztext / Bezeichnung', 'Kurztext', 'Bezeichnung', 'Description', 'Item']
            for col in df.columns:
                if any(candidate.lower() in col.lower() for candidate in desc_candidates):
                    description_col = col
                    break
            if description_col is None:
                description_col = df.columns[0]  # Use first column as fallback
        
        if article_col is None:
            # Look for common article number column names
            article_candidates = ['Artikelnummer', 'Article', 'Number', 'Code', 'ID']
            for col in df.columns:
                if any(candidate.lower() in col.lower() for candidate in article_candidates):
                    article_col = col
                    break
            if article_col is None:
                article_col = df.columns[1] if len(df.columns) > 1 else df.columns[0]
        
        print(f"Using columns: Description='{description_col}', Article='{article_col}'")
        
        # Process each row
        for idx, row in df.iterrows():
            description = str(row[description_col]).strip() if pd.notna(row[description_col]) else ""
            article_number = str(row[article_col]).strip() if pd.notna(row[article_col]) else ""
            
            # Skip empty rows, headers, and rows without article numbers
            if (description and article_number and 
                article_number != 'nan' and 
                not description.startswith('Wartung') and
                not description.lower().startswith('kurztext')):
                
                self.parse_item(description, article_number)
        
        return dict(self.tree)
    
    def parse_catalogue_data(self, data_text):
        """
        Parse the catalogue data from text format (kept for backward compatibility)
        
        Args:
            data_text (str): Raw catalogue data as text
        
        Returns:
            dict: Hierarchical tree structure
        """
        lines = data_text.strip().split('\n')
        
        for line in lines:
            if '\t' in line and line.strip():
                # Split by tab to separate description and article number
                parts = line.strip().split('\t')
                if len(parts) >= 2:
                    description = parts[0].strip()
                    article_number = parts[1].strip() if parts[1].strip() else None
                    
                    # Skip header lines and category headers without article numbers
                    if description and article_number and not description.startswith('Wartung'):
                        self.parse_item(description, article_number)
        
        return dict(self.tree)
    
    def parse_item(self, description, article_number):
        """
        Parse individual item description and create hierarchical structure
        
        Args:
            description (str): Item description
            article_number (str): Article number
        """
        # Remove leading numbers and dashes (e.g., "334 - ")
        cleaned_desc = re.sub(r'^\d+\s*-\s*', '', description).strip()
        
        # Split by " - " to get different levels
        parts = [part.strip() for part in cleaned_desc.split(' - ')]
        
        if not parts:
            return
        
        # First part is usually the main category
        main_category = parts[0]
        
        # Process the rest of the description
        if len(parts) > 1:
            # Join the remaining parts and parse them
            detail_description = ' - '.join(parts[1:])
            hierarchy = self.extract_hierarchy(detail_description)
        else:
            hierarchy = []
        
        # Build the tree structure
        self.build_tree_path(main_category, hierarchy, article_number)
    
    def extract_hierarchy(self, description):
        """
        Extract hierarchical information from item description
        
        Args:
            description (str): Detailed description
        
        Returns:
            list: List of hierarchical levels
        """
        hierarchy = []
        
        # Common patterns to identify different levels
        patterns = {
            'type': r'^([^-;]+?)(?:\s*-\s*|\s*;\s*|$)',
            'power': r'(kraftbetrieben|nicht kraftbetrieben|elektrisch|mechanisch)',
            'function': r'(mit|ohne)\s+(Brand-?/?Rauchschutzfunktion|Fluchtwegsfunktion|Brandschutzfunktion)',
            'wings': r'(\d+-flüglig)',
            'size': r'(Breite\s*[<>]\s*\d+\s*m[^;]*|Höhe\s*[<>]\s*\d+\s*m[^;]*|Einbauhöhe[^;]*|Innendurchmesser[^;]*|Dachfläche[^;]*)',
        }
        
        # Extract main type/category
        type_match = re.search(patterns['type'], description)
        if type_match:
            main_type = type_match.group(1).strip()
            if main_type and not any(x in main_type.lower() for x in ['kraftbetrieben', 'elektrisch', 'mechanisch']):
                hierarchy.append(('type', main_type))
        
        # Extract power/operation method
        power_match = re.search(patterns['power'], description)
        if power_match:
            hierarchy.append(('power', power_match.group(1)))
        
        # Extract function (Brand/Rauch/Fluchtweg)
        function_match = re.search(patterns['function'], description)
        if function_match:
            function_desc = f"{function_match.group(1)} {function_match.group(2)}"
            hierarchy.append(('function', function_desc))
        
        # Extract wings information
        wings_match = re.search(patterns['wings'], description)
        if wings_match:
            hierarchy.append(('wings', wings_match.group(1)))
        
        # Extract size information
        size_match = re.search(patterns['size'], description)
        if size_match:
            hierarchy.append(('size', size_match.group(1).strip()))
        
        # Handle special cases
        if 'Zulage' in description:
            hierarchy.append(('type', 'Zubehör'))
            if 'Feststellanlage' in description:
                hierarchy.append(('subtype', 'Feststellanlage für Brand-/Rauchschutzfunktion'))
            elif 'Motorschloss' in description:
                hierarchy.append(('subtype', 'Motorschloss'))
            elif 'Flichttürterminal' in description:
                hierarchy.append(('subtype', 'Flichttürterminal'))
            elif 'Fluchtürwächter' in description:
                hierarchy.append(('subtype', 'Fluchtürwächter'))
        
        return hierarchy
    
    def build_tree_path(self, main_category, hierarchy, article_number):
        """
        Build the tree path from main category through hierarchy to article number
        
        Args:
            main_category (str): Main category
            hierarchy (list): List of (type, value) tuples
            article_number (str): Final article number
        """
        # Ensure main category exists as a dict
        if main_category not in self.tree:
            self.tree[main_category] = {}
        elif not isinstance(self.tree[main_category], dict):
            self.tree[main_category] = {}
        
        current = self.tree[main_category]
        
        # Group hierarchy items by logical structure
        type_items = [item[1] for item in hierarchy if item[0] == 'type']
        power_items = [item[1] for item in hierarchy if item[0] == 'power']
        function_items = [item[1] for item in hierarchy if item[0] == 'function']
        wings_items = [item[1] for item in hierarchy if item[0] == 'wings']
        size_items = [item[1] for item in hierarchy if item[0] == 'size']
        subtype_items = [item[1] for item in hierarchy if item[0] == 'subtype']
        
        # Build path: type -> power -> function -> wings -> size
        path_elements = type_items + power_items + function_items + wings_items + size_items + subtype_items
        
        # Navigate/create the path
        for i, element in enumerate(path_elements):
            if element not in current:
                current[element] = {}
            elif not isinstance(current[element], dict):
                # If we hit a leaf node (string), we need to restructure
                # Save the existing value and convert to dict
                existing_value = current[element]
                current[element] = {"_direct": existing_value}
            current = current[element]
        
        # Set the article number at the end
        if path_elements:
            # Navigate back to set the final value
            target = self.tree[main_category]
            for element in path_elements[:-1]:
                target = target[element]
            
            # Check if the final element already exists
            final_element = path_elements[-1]
            if final_element in target and isinstance(target[final_element], dict):
                # If it's a dict, add the article number as a special key
                target[final_element]["_article"] = article_number
            else:
                # Direct assignment
                target[final_element] = article_number
        else:
            # Direct assignment to main category - but main category should be a dict
            # Store the article number with a special key
            if isinstance(self.tree[main_category], dict):
                self.tree[main_category]["_direct"] = article_number
            else:
                self.tree[main_category] = article_number
    
    def save_to_json(self, filename, tree_data):
        """Save the tree structure to a JSON file"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(tree_data, f, indent=2, ensure_ascii=False)
    
    def load_from_file(self, filename):
        """Load catalogue data from file"""
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()

class CustomerEquipmentMatcher:
    def __init__(self, decision_tree):
        self.decision_tree = decision_tree
        
        # Comprehensive mapping from customer Kostengruppe to catalogue categories
        
        self.kostengruppe_mapping = kostengruppe_mapping
    
    def extract_kostengruppe(self, merkmal):
        """Extract Kostengruppe from EQ-Merkmal text"""
        if pd.isnull(merkmal):
            return None
        match = re.search(r'Kostengruppe DIN 276-1:\s*(\d{3})', str(merkmal))
        if match:
            return match.group(1)
        return None
    
    def search_kostengruppe_in_all(self, row):
        """Search for Kostengruppe in all EQ-Merkmal columns"""
        for col in row.index:
            if 'EQ-Merkmal' in col:
                match = self.extract_kostengruppe(row[col])
                if match:
                    return match
        return None
    
    def extract_equipment_characteristics(self, row):
        """Extract relevant characteristics from equipment data"""
        characteristics = {
            'bezeichnung': str(row.get('EQ-Bezeichnung', '')).lower(),
            'klasse': str(row.get('EQ-Klasse-Bezeichnung', '')).lower(),
            'gewerk': str(row.get('Gewerk', '')).lower(),
            'anlagenauspragung': str(row.get('Anlagenausprägung', '')).lower(),
        }
        
        # Extract characteristics from EQ-Merkmal columns
        merkmal_data = {}
        for col in row.index:
            if 'EQ-Merkmal' in col and pd.notna(row[col]):
                merkmal_text = str(row[col]).lower()
                
                # Look for power/operation characteristics
                if any(keyword in merkmal_text for keyword in ['kraftbetrieben', 'elektrisch', 'mechanisch']):
                    if 'kraftbetrieben' in merkmal_text:
                        merkmal_data['power'] = 'kraftbetrieben'
                    elif 'elektrisch' in merkmal_text:
                        merkmal_data['power'] = 'elektrisch'
                    elif 'mechanisch' in merkmal_text:
                        merkmal_data['power'] = 'mechanisch'
                
                # Look for safety functions
                if any(keyword in merkmal_text for keyword in ['brand', 'rauch', 'flucht']):
                    if 'brand' in merkmal_text or 'rauch' in merkmal_text:
                        merkmal_data['safety'] = 'mit brand-/rauchschutzfunktion'
                    elif 'flucht' in merkmal_text:
                        merkmal_data['safety'] = 'mit fluchtwegsfunktion'
                
                # Look for size information
                if any(keyword in merkmal_text for keyword in ['breite', 'höhe', 'durchmesser', 'fläche']):
                    merkmal_data['size_info'] = merkmal_text
        
        characteristics.update(merkmal_data)
        return characteristics
    
    def find_best_match(self, kostengruppe, characteristics):
        """
        Find the best matching article number for given characteristics
        
        Args:
            kostengruppe (str): Customer equipment Kostengruppe
            characteristics (dict): Equipment characteristics
        
        Returns:
            tuple: (article_number, score) or (None, 0) if no match found
        """
        # Use the enhanced fuzzy matching
        article_number, score, path = self.find_best_match_fuzzy(kostengruppe, characteristics)
        return article_number, score
    
    def analyze_catalogue_structure(self):
        """Analyze and print the catalogue structure to help with mapping"""
        print("\n=== CATALOGUE STRUCTURE ANALYSIS ===")
        print("Available categories in catalogue:")
        
        for category, content in self.decision_tree.tree.items():
            print(f"\n📁 {category}")
            if isinstance(content, dict):
                self._print_subcategories(content, indent="  ")
            else:
                print(f"  → {content}")
    
    def _print_subcategories(self, node, indent="", max_depth=2, current_depth=0):
        """Helper to print subcategories with limited depth"""
        if current_depth >= max_depth:
            return
            
        for key, value in list(node.items())[:5]:  # Limit to first 5 items
            if key.startswith('_'):
                if key in ['_article', '_direct']:
                    print(f"{indent}→ Article: {value}")
            else:
                print(f"{indent}├── {key}")
                if isinstance(value, dict):
                    self._print_subcategories(value, indent + "│   ", max_depth, current_depth + 1)
                else:
                    print(f"{indent}│   → {value}")
        
        if len(node) > 5:
            print(f"{indent}... and {len(node) - 5} more items")
    
    def find_best_match_fuzzy(self, kostengruppe, characteristics):
        """Enhanced matching with fuzzy search across all categories"""
        
        best_match = None
        highest_score = 0
        best_path = []
        
        # If no direct mapping, search all categories
        possible_categories = self.kostengruppe_mapping.get(kostengruppe, [])
        if not possible_categories:
            possible_categories = list(self.decision_tree.tree.keys())
        
        print(f"  Searching in categories: {possible_categories[:3]}{'...' if len(possible_categories) > 3 else ''}")
        
        for category in possible_categories:
            if category in self.decision_tree.tree:
                matches = self._search_category_for_matches_enhanced(category, characteristics)
                for match, score, path in matches:
                    if score > highest_score:
                        highest_score = score
                        best_match = match
                        best_path = path
        
        return best_match, highest_score, best_path
    
    def _search_category_for_matches_enhanced(self, category, characteristics):
        """Enhanced search with better scoring"""
        matches = []
        category_tree = self.decision_tree.tree[category]
        
        def search_recursive(node, path, score):
            if isinstance(node, str):
                # Found an article number
                matches.append((node, score, path.copy()))
            elif isinstance(node, dict):
                for key, value in node.items():
                    if key.startswith('_'):
                        # Handle special keys
                        if key in ["_article", "_direct"]:
                            matches.append((value, score, path.copy()))
                    else:
                        # Calculate score based on keyword matching
                        key_lower = key.lower()
                        new_score = score
                        
                        # Equipment type matching (high priority)
                        equipment_types = ['fenster', 'tür', 'tor', 'rolladen', 'jalousie', 'markise', 
                                         'lüftung', 'heizung', 'pumpe', 'beleuchtung', 'schalter']
                        for eq_type in equipment_types:
                            if eq_type in characteristics['bezeichnung'] and eq_type in key_lower:
                                new_score += 30
                        
                        # Power/operation method matching
                        if 'power' in characteristics:
                            power_keywords = ['kraftbetrieben', 'elektrisch', 'mechanisch', 'automatisch']
                            if any(pw in key_lower for pw in power_keywords):
                                if characteristics['power'] in key_lower:
                                    new_score += 25
                                else:
                                    new_score += 10  # Partial match
                        
                        # Safety function matching  
                        if 'safety' in characteristics:
                            safety_keywords = ['brand', 'rauch', 'flucht', 'sicherheit']
                            if any(sw in key_lower for sw in safety_keywords):
                                new_score += 20
                        
                        # Generic keyword matching
                        bezeichnung_words = characteristics['bezeichnung'].split()
                        key_words = key_lower.split()
                        common_words = set(bezeichnung_words) & set(key_words)
                        new_score += len(common_words) * 5
                        
                        # Continue search
                        search_recursive(value, path + [key], new_score)
        
        search_recursive(category_tree, [category], 0)
        return matches
    
    def _search_category_for_matches(self, category, characteristics):
        """Search within a category for matching items"""
        matches = []
        category_tree = self.decision_tree.tree[category]
        
        def search_recursive(node, path, score):
            if isinstance(node, str):
                # Found an article number
                matches.append((node, score))
            elif isinstance(node, dict):
                for key, value in node.items():
                    if key.startswith('_'):
                        # Handle special keys
                        if key in ["_article", "_direct"]:
                            matches.append((value, score))
                    else:
                        # Calculate score based on keyword matching
                        key_lower = key.lower()
                        new_score = score
                        
                        # Check for keyword matches
                        if any(char in characteristics['bezeichnung'] for char in key_lower.split()):
                            new_score += 10
                        
                        if 'power' in characteristics and characteristics['power'] in key_lower:
                            new_score += 20
                        
                        if 'safety' in characteristics and any(safety_word in key_lower for safety_word in characteristics['safety'].split()):
                            new_score += 15
                        
                        # Check for equipment type matches
                        equipment_keywords = ['fenster', 'tür', 'tor', 'rolladen', 'jalousie', 'markise']
                        for keyword in equipment_keywords:
                            if keyword in characteristics['bezeichnung'] and keyword in key_lower:
                                new_score += 25
                        
                        search_recursive(value, path + [key], new_score)
        
        search_recursive(category_tree, [category], 0)
        return matches
    
    def assign_article_numbers(self, customer_df):
        """Assign article numbers to customer equipment data"""
        
        # First extract Kostengruppe for all rows
        customer_df['Extracted_Kostengruppe'] = customer_df.apply(self.search_kostengruppe_in_all, axis=1)
        
        # Initialize columns for results
        customer_df['Artikelnummer_Assigned'] = None
        customer_df['Match_Score'] = 0
        customer_df['Match_Category'] = None
        customer_df['Match_Path'] = None
        
        successful_matches = 0
        total_rows = len(customer_df)
        
        print(f"Processing {total_rows} equipment records...")
        
        for idx, row in customer_df.iterrows():
            kostengruppe = row['Extracted_Kostengruppe']
            
            if kostengruppe:
                # Extract equipment characteristics
                characteristics = self.extract_equipment_characteristics(row)
                
                # Find best match
                article_number, score = self.find_best_match(kostengruppe, characteristics)
                
                if article_number:
                    customer_df.at[idx, 'Artikelnummer_Assigned'] = article_number
                    customer_df.at[idx, 'Match_Score'] = score
                    customer_df.at[idx, 'Match_Category'] = kostengruppe
                    successful_matches += 1
                    
                    if idx % 100 == 0:  # Progress indicator
                        print(f"Processed {idx}/{total_rows} rows...")
        
        print(f"\nMatching complete!")
        print(f"Successfully matched: {successful_matches}/{total_rows} ({successful_matches/total_rows*100:.1f}%)")
        
        return customer_df
    
    def get_matching_summary(self, customer_df):
        """Get summary of matching results"""
        if 'Artikelnummer_Assigned' not in customer_df.columns:
            print("No matching results found. Run assign_article_numbers first.")
            return
        
        total = len(customer_df)
        matched = customer_df['Artikelnummer_Assigned'].notna().sum()
        unmatched = total - matched
        
        print(f"\n=== MATCHING SUMMARY ===")
        print(f"Total equipment records: {total}")
        print(f"Successfully matched: {matched} ({matched/total*100:.1f}%)")
        print(f"Unmatched: {unmatched} ({unmatched/total*100:.1f}%)")
        
        # Show matches by Kostengruppe
        if matched > 0:
            print(f"\nMatches by Kostengruppe:")
            kostengruppe_summary = customer_df[customer_df['Artikelnummer_Assigned'].notna()]['Extracted_Kostengruppe'].value_counts()
            for kg, count in kostengruppe_summary.items():
                print(f"  {kg}: {count} matches")
        
        # Show some examples
        if matched > 0:
            print(f"\nExample matches:")
            examples = customer_df[customer_df['Artikelnummer_Assigned'].notna()][
                ['EQ-Bezeichnung', 'Extracted_Kostengruppe', 'Artikelnummer_Assigned', 'Match_Score']
            ].head(5)
            print(examples.to_string(index=False))
        
        # Show unmatched items with Kostengruppe
        if unmatched > 0:
            print(f"\nUnmatched items by Kostengruppe:")
            unmatched_kg = customer_df[customer_df['Artikelnummer_Assigned'].isna()]['Extracted_Kostengruppe'].value_counts()
            for kg, count in unmatched_kg.items():
                print(f"  {kg}: {count} unmatched")

# Example usage for customer matching
def match_customer_equipment(catalogue_file_path, customer_df):
    """
    Complete workflow to match customer equipment with catalogue
    
    Args:
        catalogue_file_path (str): Path to catalogue Excel file
        customer_df (pd.DataFrame): Customer equipment DataFrame
    
    Returns:
        pd.DataFrame: Customer DataFrame with assigned article numbers
    """
    
    # Step 1: Load and parse catalogue
    print("Step 1: Loading catalogue...")
    try:
        catalogue_df = pd.read_excel(catalogue_file_path)
        parser = CatalogueParser()
        tree_data = parser.parse_catalogue_dataframe(catalogue_df)
        
        # Create decision tree
        decision_tree = CatalogueDecisionTree(tree_data)
        print("Catalogue loaded and parsed successfully!")
        
    except Exception as e:
        print(f"Error loading catalogue: {e}")
        return customer_df
    
    # Step 2: Create matcher and assign article numbers
    print("\nStep 2: Matching customer equipment...")
    matcher = CustomerEquipmentMatcher(decision_tree)
    result_df = matcher.assign_article_numbers(customer_df.copy())
    
    # Step 3: Show summary
    matcher.get_matching_summary(result_df)
    
    return result_df

# Example usage and testing
if __name__ == "__main__":
    
    # Method 1: Complete customer equipment matching workflow
    print("=== CUSTOMER EQUIPMENT MATCHING ===")
    
    catalogue_file_path = 'EP_Katalog.xlsx' 
    
    # If you have customer_df already loaded:
    # result_df = match_customer_equipment(catalogue_file_path, customer_df)
    
    # Or load customer data from Excel:
    customer_df = pd.read_excel('Kundendatei.xlsx')
    result_df = match_customer_equipment(catalogue_file_path, customer_df)
    


=== CUSTOMER EQUIPMENT MATCHING ===
Step 1: Loading catalogue...
Using columns: Description='Kurztext / Bezeichnung', Article='Artikelnummer'
Catalogue loaded and parsed successfully!

Step 2: Matching customer equipment...
Processing 973 equipment records...
  Searching in categories: ['Wärmeversorgungsanlagen']
  Searching in categories: ['Wärmeversorgungsanlagen']
  Searching in categories: ['Wärmeversorgungsanlagen']
  Searching in categories: ['Anlagen der Außenanlagen']
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Nutzungsspezifische Anlagen']
  Searching in categories: ['Gebäudeautomation']
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', '

  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Anlagen der Außenanlagen']
  Searching in categories: ['Anlagen der Außenanlagen']
  Searching in categories: ['Anlagen der Außenanlagen']
  Searching in categories: ['Anlagen der Außenanlagen']
  Searching in categories: ['Anlagen der Außenanlagen']
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfe

  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Gebäudeautomation']
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Nutzungsspezifische Anlagen']
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Wärmeversorgungsanlagen']
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Fa

  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp-

  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp-

  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Gebäudeautomation']
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Wärmeversorgungsanlagen']
  Searching in categories: ['Gebäudeautomation']
  Searching in categories: ['Gebäudeautomation']
  Searching in categories: ['Gebäudeautomation']
  Searc

  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp-

  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Wärmeversorgungsanlagen']
  Searching in categories: ['Wärmeversorgungsanlagen']
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Wärmeversorgu

  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp-

  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Fenster mit Kipp- oder Drehflügel', 'Hochwasserschutzfenster', 'Falttor']...
  Searching in categories: ['Wärmeversorgungsa

In [45]:
print(result_df['Artikelnummer_Assigned'].unique())

[None '474.01.04.137.00.00' '334.03.01.024.00.00' '334.03.01.000.05.01'
 '334.03.01.000.05.02' '389.01.01.000.01.00' '334.03.01.181.00.00'
 '387.01.01.000.00.01' '411.04.02.000.00.02' '449.01.01.000.04.00'
 '387.01.01.000.00.02' '461.01.01.000.01.08' '362.01.01.000.01.02'
 '461.01.01.000.01.10' '334.02.01.026.00.00' '334.02.01.000.06.01'
 '474.01.02.000.00.00' '474.01.01.000.00.00' '421.03.02.000.08.01'
 '338.01.01.194.01.00']


In [47]:
print(result_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 66 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   WirtEinh                973 non-null    int64  
 1   EQ_übergeordnet         364 non-null    float64
 2   Equipment               973 non-null    int64  
 3   EQ-Bezeichnung          973 non-null    object 
 4   EQ-Klasse               973 non-null    int64  
 5   EQ-Klasse-Bezeichnung   973 non-null    object 
 6   Gewerk                  973 non-null    object 
 7   Anlagenausprägung       973 non-null    object 
 8   EQ-Menge                973 non-null    int64  
 9   EQ-Merkmal_001          506 non-null    object 
 10  EQ-Merkmal_002          321 non-null    object 
 11  EQ-Merkmal_003          627 non-null    object 
 12  EQ-Merkmal_004          681 non-null    object 
 13  EQ-Merkmal_005          419 non-null    object 
 14  EQ-Merkmal_006          378 non-null    ob

**Note: Fuzzy match is NOT enough.**